In [1]:
!pip install pandas
!pip install transformers
!pip install accelerate -U
!pip install datasets
!pip install scikit-learn

In [2]:
import os
import sys
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import AutoFeatureExtractor, WhisperForAudioClassification, Trainer, TrainingArguments
import datetime
from sklearn.metrics import accuracy_score
# sys.path.append("E:/university/FYT/repos/multi_modal_ser")
sys.path.append("/home/multi_modal_ser")
from utils.dataset import MMSERDataset
from datasets import load_metric
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.utils.data import Dataset, DataLoader
import torch
from torch.utils.data import Dataset, Subset

### Log the Notebook

In [3]:
import sys
import logging

nblog = open("hubert_nb.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [4]:
print(device)
!nvidia-smi

cuda
Sat Oct 14 15:11:06 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.05              Driver Version: 535.86.05    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:01:00.0 Off |                  Off |
|  0%   36C    P8              29W / 450W |      5MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+--------------------

In [5]:
# mmser_ds = torch.load("E:/datasets/preprocessed/dataset/mmser_ds.pt")
mmser_ds = torch.load("/home/mmser_ds.pt")
print("Emotion ID: ", mmser_ds.df_["emotion_id"].unique())

Emotion ID:  [2. 1. 3. 0.]


### HUBERT

In [6]:
MODEL_NAME = "facebook/hubert-large-ls960-ft"

In [7]:
from transformers import AutoProcessor, HubertModel
from tqdm import tqdm

class ProcessedDataset(Dataset):
    
    def __init__(self, base_ds, pretrained_model):
        self.base_ds = base_ds
        self.processor = AutoProcessor.from_pretrained(pretrained_model)
        self.__process__()
        
    def __process__(self):
        self.input_values_list = []
        self.attention_mask_list = []
        for raw_audio in tqdm(self.base_ds.raw_list):
            processed = self.processor(raw_audio, 
                                       sampling_rate=16000,
                                       padding='max_length',
                                       max_length=300000,
                                       truncation=True, 
                                      return_tensors="np")
            self.input_values_list.append(processed["input_values"].squeeze())
            self.attention_mask_list.append(processed["attention_mask"].squeeze())
    def __len__(self):
        return len(self.base_ds)
    
    def __getitem__(self, idx):
        base_dict = self.base_ds[idx]
        base_dict["input_values"] = self.input_values_list[idx]
        base_dict["attention_mask"] = self.attention_mask_list[idx]
        del base_dict["audio"]
        return base_dict
        
        

In [8]:
AutoProcessor.from_pretrained(MODEL_NAME)
processed_ds = ProcessedDataset(mmser_ds, MODEL_NAME)

100%|██████████| 5531/5531 [00:09<00:00, 577.42it/s]


### Build Model

In [9]:
from transformers import AutoProcessor, HubertModel, AutoModel
from datasets import load_dataset

processor = AutoProcessor.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.config.use_weighted_layer_sum

Some weights of HubertModel were not initialized from the model checkpoint at facebook/hubert-large-ls960-ft and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


False

In [10]:
class CustomClassifier(torch.nn.Module):

    def __init__(self, pretrained_model, num_labels=4):
        super(CustomClassifier, self).__init__()
        self.encoder = AutoModel.from_pretrained(pretrained_model)
        self.config = self.encoder.config
        self.config.num_labels = num_labels
        self.projector = nn.Linear(self.config.hidden_size, self.config.classifier_proj_size)
        self.classifier = nn.Linear(self.config.classifier_proj_size, num_labels)

    def forward(
        self,
        input_values,
        attention_mask = None,
        output_attentions = None,
        output_hidden_states = None,
        return_dict = None,
        labels = None,
    ):
        outputs = self.encoder(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        hidden_states = outputs[0]

        hidden_states = self.projector(hidden_states)
        if attention_mask is None:
            pooled_output = hidden_states.mean(dim=1)
        else:
            padding_mask = self.encoder._get_feature_vector_attention_mask(hidden_states.shape[1], attention_mask)
            hidden_states[~padding_mask] = 0.0
            pooled_output = hidden_states.sum(dim=1) / padding_mask.sum(dim=1).view(-1, 1)
        logits = self.classifier(pooled_output)
        return {"logits":logits}
    
model = CustomClassifier(MODEL_NAME, mmser_ds.df_["emotion_id"].nunique())

Some weights of HubertModel were not initialized from the model checkpoint at facebook/hubert-large-ls960-ft and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# logits = model(torch.Tensor(processed_ds[:2]["input_values"]).squeeze(), 
#                       torch.Tensor(processed_ds[:2]["attention_mask"]).squeeze()).last_hidden_state
# print(logits.shape)

### Split Dataset

In [12]:
meta_df_ = processed_ds.base_ds.df_
sess_dict = meta_df_.groupby("session").groups
all_indices = set(meta_df_.index.tolist())

In [13]:
sess_ds = {}
for sess in sess_dict:
    sess_ds[sess+"_train"] = Subset(processed_ds, 
                                    indices=list(all_indices-set(sess_dict[sess])))
    sess_ds[sess+"_test"] = Subset(processed_ds, 
                                    indices=sess_dict[sess])
    

In [14]:
processed_ds[1]

{'sess': 'Ses01F',
 'fn': 'Ses01F_impro01_F001',
 'text': 'Yeah.',
 'labels': 2.0,
 'input_values': array([ 0.11388988,  0.11078218, -0.06635647, ...,  0.        ,
         0.        ,  0.        ]),
 'attention_mask': array([1, 1, 1, ..., 0, 0, 0], dtype=int32)}

### Custom Trainer, Metrics

In [15]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").type(torch.LongTensor).to(device)
        
        input_values = inputs["input_values"].to(device).to(torch.float32)
        attention_mask = inputs["attention_mask"].to(device).to(torch.float32)
        outputs = model(input_values, 
                       attention_mask)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss() 
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))        
        return (loss, outputs) if return_outputs else loss
    
def weighted_acc(y_true, y_pred):
    return np.sum((np.array(y_pred).ravel() == np.array(y_true).ravel()))*1.0/len(y_true)
    
def unweighted_acc(y_true, y_pred):
    y_true = np.array(y_true).ravel()
    y_pred = np.array(y_pred).ravel()
    classes = np.unique(y_true)
    classes_accuracies = np.zeros(classes.shape[0])
    for num, cls in enumerate(classes):
        classes_accuracies[num] = weighted_acc(y_true[y_true == cls], y_pred[y_true == cls])
    return np.mean(classes_accuracies)

def compute_metrics(eval_preds):
    logits, labels = eval_preds.predictions, eval_preds.label_ids
    predictions = np.argmax(logits, axis=-1)

    metric_f1 = load_metric("f1")
    metric_acc = load_metric("accuracy")
    logits, labels = eval_preds.predictions, eval_preds.label_ids
    predictions = np.argmax(logits, axis=-1)
    f1_ = metric_f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    acc_ = metric_acc.compute(predictions=predictions, references=labels)["accuracy"]
    
    return {"wa":weighted_acc(labels, predictions), 
            "ua":unweighted_acc(labels, predictions),
            "f1":f1_, 
            "accuracy":acc_}

### Set SESS_ID

In [16]:
SESS_ID = list(sess_dict.keys())[0]

def build_ds(sess_id):
    train_size = int(len(sess_ds[sess_id+"_train"])*0.75)
    val_size = len(sess_ds[sess_id+"_train"])-train_size
    train_set, val_set = torch.utils.data.random_split(sess_ds[sess_id+"_train"], [train_size, val_size])
    test_set = sess_ds[sess_id+"_test"]

    print("Train Samples:", len(train_set))
    print("Val Samples:", len(val_set))
    print("Test Samples:", len(test_set))
    
    return train_set, val_set, test_set

In [17]:
train_set, val_set, test_set = build_ds(SESS_ID)

Train Samples: 3768
Val Samples: 1256
Test Samples: 507


##### Load Model

In [24]:
model.load_state_dict(torch.load(output_dir+"/pytorch_model.bin"))

<All keys matched successfully>

##### Freeze

In [25]:
for param in model.encoder.parameters():
    param.requires_grad = False

In [26]:
output_dir=os.path.join("/home/multi_modal_ser/finetune_encoder/check_pts", "HUBERT", SESS_ID, datetime.datetime.now().date().strftime(format="%Y-%m-%d"))

training_args = TrainingArguments(output_dir)
training_args.logging_steps = 50
training_args.eval_steps = 50
training_args.remove_unused_columns=False
training_args.per_device_train_batch_size=4
training_args.per_device_eval_batch_size=16
training_args.evaluation_strategy="steps" 
training_args.logging_strategy="steps"
training_args.load_best_model_at_end=True,
training_args.save_strategy = "no"
training_args.learning_rate=1e-4
training_args.num_train_epochs=2

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
)

In [27]:
trainer.train()

Step,Training Loss,Validation Loss,Wa,Ua,F1,Accuracy
50,1.043900,1.236608,0.445860,0.477714,0.419069,0.445860
100,0.994800,1.040746,0.535828,0.566848,0.522122,0.535828
150,0.992100,1.252210,0.445860,0.486691,0.413902,0.445860
200,0.934700,1.180189,0.482484,0.519043,0.476113,0.482484
250,0.971800,1.085114,0.519904,0.542739,0.485675,0.519904
300,1.001700,1.068771,0.519904,0.550307,0.508779,0.519904
350,0.990900,1.094590,0.519904,0.548391,0.512292,0.519904
400,0.958100,1.125435,0.509554,0.538087,0.502113,0.509554
450,0.984400,1.104307,0.517516,0.538026,0.508919,0.517516
500,1.011100,1.118384,0.511146,0.541752,0.503065,0.511146


TrainOutput(global_step=942, training_loss=0.9765039391325284, metrics={'train_runtime': 1274.2128, 'train_samples_per_second': 5.914, 'train_steps_per_second': 0.739, 'total_flos': 0.0, 'train_loss': 0.9765039391325284, 'epoch': 2.0})

In [28]:
trainer.save_model(output_dir)